<a href="https://colab.research.google.com/github/Mhuna91/10-08-2022_Assignment/blob/main/Tokenization_of_Qualitative_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()


Saving Advanced Valuation.csv to Advanced Valuation (1).csv


In [3]:
!pip install scikit-learn==1.2.2 pyLDAvis==3.4.1 vaderSentiment wordcloud pandas matplotlib


In [1]:
!pip install scikit-learn==1.2.2 pyLDAvis==3.4.1 vaderSentiment wordcloud pandas matplotlib


In [1]:
from sklearn.feature_extraction.text import CountVectorizer
if not hasattr(CountVectorizer, 'get_feature_names'):
    CountVectorizer.get_feature_names = CountVectorizer.get_feature_names_out


In [7]:
import pandas as pd

# Upload the CSV again
from google.colab import files
uploaded = files.upload()

# Replace the file name if different
df = pd.read_csv("Advanced Valuation.csv")


Saving Advanced Valuation.csv to Advanced Valuation (2).csv


In [8]:
# Combine relevant qualitative columns
df['combined'] = df.iloc[:, 8].fillna('') + ' ' + df.iloc[:, 9].fillna('')

# Clean text
df['cleaned'] = df['combined'].str.lower().str.replace(r'\W', ' ', regex=True).str.strip()


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = CountVectorizer(stop_words='english', min_df=2)
dtm = vectorizer.fit_transform(df['cleaned'])

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(dtm)


LatentDirichletAllocation(n_components=5, random_state=42)

In [10]:
from pyLDAvis import prepare
import pyLDAvis
pyLDAvis.enable_notebook()  # Use pyLDAvis.show(panel) if not in notebook

panel = prepare(
    topic_term_dists=lda.components_ / lda.components_.sum(axis=1)[:, None],
    doc_topic_dists=lda.transform(dtm),
    doc_lengths=(dtm > 0).sum(axis=1).A1,
    vocab=vectorizer.get_feature_names_out(),
    term_frequency=dtm.sum(axis=0).A1
)

pyLDAvis.display(panel)


In [11]:
import numpy as np

# Get topic probabilities for each document
doc_topic_probs = lda.transform(dtm)

# Assign each document to the topic with the highest probability
dominant_topics = np.argmax(doc_topic_probs, axis=1)

# Add to your original dataframe
df['Dominant_Topic'] = dominant_topics

# OPTIONAL: Map topic numbers to readable labels
topic_labels = {
    0: 'Policy & Curriculum Reform',
    1: 'Valuers’ Training & Application',
    2: 'CPD, Standards & Education',
    3: 'Professional Advancement & Awareness',
    4: 'Practical Awareness & Adoption'
}
df['Topic_Label'] = df['Dominant_Topic'].map(topic_labels)

# Preview result
df[['cleaned', 'Dominant_Topic', 'Topic_Label']].head()


,cleaned,Dominant_Topic,Topic_Label
0,msc 0,0,Policy & Curriculum Reform
1,bsc 0,1,Valuers’ Training & Application
2,msc 0,0,Policy & Curriculum Reform
3,msc 0,0,Policy & Curriculum Reform
4,msc 0,0,Policy & Curriculum Reform


In [12]:
# Save labeled data to a new file
df.to_csv("Labeled_Valuation_Responses.csv", index=False)

# For Google Colab: download the file
from google.colab import files
files.download("Labeled_Valuation_Responses.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Apply to each cleaned response
df['Sentiment'] = df['cleaned'].apply(lambda text: analyzer.polarity_scores(text) if pd.notnull(text) else None)

# Expand the dictionary into separate columns
sentiment_df = df['Sentiment'].apply(pd.Series)
df = pd.concat([df, sentiment_df], axis=1)

# Preview
df[['cleaned', 'Topic_Label', 'compound', 'pos', 'neu', 'neg']].head()


,cleaned,Topic_Label,compound,pos,neu,neg
0,msc 0,Policy & Curriculum Reform,0.0,0.0,1.0,0.0
1,bsc 0,Valuers’ Training & Application,0.0,0.0,1.0,0.0
2,msc 0,Policy & Curriculum Reform,0.0,0.0,1.0,0.0
3,msc 0,Policy & Curriculum Reform,0.0,0.0,1.0,0.0
4,msc 0,Policy & Curriculum Reform,0.0,0.0,1.0,0.0


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('cleaned').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Topic_Label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['compound']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'compound'}, axis=1)
              .sort_values('compound', ascending=True))
  xs = counted['compound']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('compound', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('cleaned')):
  _plot_series(series, series_name, i)
  fig.legend(title='cleaned', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('compound')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['compound']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'compound'}, axis=1)
              .sort_values('compound', ascending=True))
  xs = counted['compound']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('compound', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Topic_Label')):
  _plot_series(series, series_name, i)
  fig.legend(title='Topic_Label', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('compound')
_ = plt.ylabel('count()')

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Topic_Label'].value_counts()
    for x_label, grp in _df_15.groupby('cleaned')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('cleaned')
_ = plt.ylabel('Topic_Label')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['cleaned'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x='index', y='cleaned', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_17['Topic_Label'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_17, x='index', y='Topic_Label', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)